In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
import time
import random
from collections import deque # Efficiëntere queue dan een lijst

In [2]:
# --- Configuratie ---
INFINITE_CRAFT_URL = "https://neal.fun/infinite-craft/"
INITIAL_ELEMENTS = ["Water", "Fire", "Wind", "Earth"]
WAIT_TIMEOUT = 0.2  # Seconden wachttijd voor elementen
ACTION_DELAY = 0 # Kleine pauze tussen acties
COMBINATION_DELAY = 0.5 # Wachttijd na een combinatie om resultaat te zien

# --- Data Opslag ---
discovered_elements = set(INITIAL_ELEMENTS)
recipes = {} # Dict om recepten op te slaan: {(elem1, elem2): result}
elements_to_process = deque(INITIAL_ELEMENTS) # Queue van elementen om te proberen te combineren
tried_combinations = set() # Set van (sorted_tuple) van reeds geprobeerde combinaties

In [3]:
# --- Hoofdlogica ---
options = uc.ChromeOptions()
    # options.add_argument("--headless") # Headless kan problemen geven met detectie/interactie

# Specify your installed Chrome major version (135 in this case)
chrome_major_version = 135
driver = uc.Chrome(options=options, use_subprocess=True, version_main=chrome_major_version)

driver.get(INFINITE_CRAFT_URL)

In [8]:
# can we find //*[@id="instances"] in the page? and can we print all the elements in it?
try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="instances"]')))
    elements = driver.find_elements(By.XPATH, '//*[@id="instances"]/*')
    for element in elements:
        print(element.text)
except TimeoutException:
    print("Element not found within the given time.")
    driver.quit()
    exit(1)


Wind
🌱
Plant
💨
Steam


In [9]:
# find all the elements in the sidbar //*[@id="sidebar"]/div[3]/div
try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="sidebar"]/div[3]/div')))
    elements = driver.find_elements(By.XPATH, '//*[@id="sidebar"]/div[3]/div/*')
    for element in elements:
        print(element.text)
except TimeoutException:
    print("Element not found within the given time.")
    driver.quit()
    exit(1)

💧 Water
🔥 Fire
🌬️ Wind
🌍 Earth
🌱 Plant
💨 Steam
🌳 Tree
🌲 Forest
🌲 Wood
📃 Paper
📚 Book
📚 Bookshelf
📚 Library
💡 Knowledge




In [ ]:
# find the element in the hand //*[@id="instances-top"]
time.sleep(3)
try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="instances-top"]')))
    element = driver.find_element(By.XPATH, '//*[@id="instances-top"]')
    print(element.text)
except TimeoutException:
    print("Element not found within the given time.")
    driver.quit()
    exit(1)

📚
Bookshelf


In [ ]:
print(f"Starten met combineren. Elementen te verwerken: {len(elements_to_process)}")
time.sleep(2) # Extra pauze voor stabiliteit
# Hoofdlus: verwerk elementen uit de queue
processed_count = 0

elements_to_process = deque(list(discovered_elements)) # Reset queue met ontdekte elementen

while elements_to_process:
    elem1 = elements_to_process.popleft() # Pak het volgende element
    print(f"\n--- Verwerken Combinaties met: {elem1} ---")

    # Maak een kopie van de huidige bekende elementen om mee te combineren
    # Dit voorkomt problemen als de set tijdens de loop verandert
    elements_to_combine_with = list(discovered_elements)
    #random.shuffle(elements_to_combine_with) # Random volgorde kan helpen vastlopen te voorkomen
    discovered_elements_new = discovered_elements.copy() # Nieuwe set voor deze iteratie

    for elem2 in elements_to_combine_with:
       # Maak een gesorteerd tuple voor de 'tried' check (A+B is zelfde als B+A)
        combo_pair = tuple(sorted((elem1, elem2)))

            # Sla over als we deze combinatie al geprobeerd hebben of elementen gelijk zijn
        if elem1 == elem2 or combo_pair in tried_combinations:
            continue

            # Markeer als geprobeerd
        tried_combinations.add(combo_pair)

            # Probeer de combinatie
        result = combine_elements(driver, elem1, elem2)

            # Als er een nieuw resultaat is:
        if result and result not in discovered_elements_new:
            discovered_elements_new.add(result)
            elements_to_process.append(result) # Voeg toe aan queue voor latere combinaties
            recipes[combo_pair] = result
            print(f"Nieuw element toegevoegd aan queue: {result}. Queue size: {len(elements_to_process)}")
            print(f"Totaal ontdekte elementen: {len(discovered_elements_new)}")

        # Kleine pauze om niet te snel te gaan
        time.sleep(ACTION_DELAY)

    processed_count += 1
    print(f"--- Einde combinaties voor {elem1}. {processed_count} elementen verwerkt. ---")

discovered_elements = discovered_elements_new # Update de globale set met ontdekte elementen


In [ ]:
print("\n--- Script Klaar ---")
print(f"Totaal ontdekte elementen: {len(discovered_elements)}")
# print(f"Elementen: {sorted(list(discovered_elements))}")
print(f"\nGevonden Recepten ({len(recipes)}):")
# Sorteer recepten op resultaat voor leesbaarheid
sorted_recipes = sorted(recipes.items(), key=lambda item: item[1])
for pair, result in sorted_recipes:
    print(f"  {pair[0]} + {pair[1]} = {result}")

if driver:
    print("\nBrowser sluiten...")
    driver.quit()
    print("Browser gesloten.")

In [ ]:
import csv
import os

# Define the folder name
folder_name = 'recipes'

# Create the folder if it doesn't exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Define the CSV file path
csv_file_path = os.path.join(folder_name, 'discovered_recipes.csv')

# Write the recipes to the CSV file
try:
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Element1', 'Element2', 'Result']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for (elem1, elem2), result in recipes.items():
            writer.writerow({'Element1': elem1, 'Element2': elem2, 'Result': result})
    print(f"Recipes successfully saved to {csv_file_path}")
except Exception as e:
    print(f"Error writing recipes to CSV: {e}")

In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv('recipes/discovered_recipes.csv')
combinations = df.to_numpy().tolist()

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Convert the dictionary to a DataFrame
#df = pd.DataFrame.from_dict(df, orient='index', columns=['Element1', 'Element2', 'Result']).reset_index(drop=True)

# Create directed graph
G = nx.DiGraph()
for _, row in df.iterrows():
    e1, e2, result = row['Element1'], row['Element2'], row['Result']
    G.add_node(e1)
    G.add_node(e2)
    G.add_node(result)
    G.add_edge(e1, result)
    G.add_edge(e2, result)

# Compute node sizes based on degree (number of connections)
degrees = dict(G.degree())
# Scale sizes: base size plus multiplier
node_sizes = [300 + degrees[n] * 300 for n in G.nodes()]

# Layout and draw
plt.figure(figsize=(12, 12))
pos = nx.spring_layout(G, k=1.2, seed=42)
nx.draw(G, pos, with_labels=True, node_size=node_sizes, font_size=10, arrowsize=15)
plt.title("Infinite Craft Combination Network (Node Size ∝ Degree)", pad=20)
plt.axis('off')
plt.show()
